In [0]:
# if gsync_save:
#     try:
#         import utils
#     except ModuleNotFoundError:
#         !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
#         import utils

#     gsync = utils.GDriveSync()
# try:
#     from allconv import AllConvNet
# except ModuleNotFoundError: 
#     !wget https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
#     from allconv import AllConvNet

In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms

In [16]:
import torch.nn as nn
import torch.nn.functional as F

class AllConvNet(nn.Module):
    def __init__(self, input_size, n_classes=10, **kwargs):
        super(AllConvNet, self).__init__()
        self.conv1 = nn.Conv2d(input_size, 96, 5, padding=1)
        self.conv2 = nn.Conv2d(96, 96, 1, padding=1)
        self.maxPool3=nn.MaxPool2d(3,stride=2,padding=1)
        self.conv4 = nn.Conv2d(96, 192, 5, padding=1)
        self.conv5 = nn.Conv2d(192, 192, 1, padding=1)
        self.maxPool6=nn.MaxPool2d(3,stride=2,padding=1)
        self.conv7 = nn.Conv2d(192, 192, 3, padding=1)
        self.conv8 = nn.Conv2d(192, 192, 1)
        self.class_conv = nn.Conv2d(192, n_classes, 1)
    
    def forward(self, x):
        x_drop = F.dropout(x, .2)
        conv1_out = F.relu(self.conv1(x_drop))
        conv2_out = F.relu(self.conv2(conv1_out))
        maxPool3_out = F.relu(self.maxPool3(conv2_out))
        maxPool3_out_drop = F.dropout(maxPool3_out, .5)
        conv4_out = F.relu(self.conv4(maxPool3_out_drop))
        conv5_out = F.relu(self.conv5(conv4_out))
        maxPool6_out = F.relu(self.maxPool6(conv5_out))
        maxPool6_out_drop = F.dropout(maxPool6_out, .5)
        conv7_out = F.relu(self.conv7(maxPool6_out_drop))
        conv8_out = F.relu(self.conv8(conv7_out))

        class_out = F.relu(self.class_conv(conv8_out))
        pool_out = F.adaptive_avg_pool2d(class_out, 1)
        pool_out.squeeze_(-1)
        pool_out.squeeze_(-1)
        return pool_out

trial=AllConvNet(3)
print(trial)

AllConvNet(
  (conv1): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (maxPool3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv4): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (maxPool6): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (class_conv): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
)


In [17]:
cuda =torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
#train_mean = trainset.train_data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
#train_std = trainset.train_data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
model = AllConvNet(3)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        # if gsync_save:
        #     gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [20]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.300353
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.298036
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.305791
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.301978
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.310738
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.307308
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.302328
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.300628
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.306128
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.298060
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.302081
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.308060
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.310849
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.304328
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.303001
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.296664

Test set: Average loss: 0.0186, Accuracy: 1038/10000 (10%)



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type AllConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Streaming output truncated to the last 5000 lines.
Test set: Average loss: 0.0075, Accuracy: 6767/10000 (68%)

Train Epoch: 87 [0/50000 (0%)]	Loss: 0.960920
Train Epoch: 87 [3200/50000 (6%)]	Loss: 1.079472
Train Epoch: 87 [6400/50000 (13%)]	Loss: 0.808160
Train Epoch: 87 [9600/50000 (19%)]	Loss: 1.102743
Train Epoch: 87 [12800/50000 (26%)]	Loss: 0.788290
Train Epoch: 87 [16000/50000 (32%)]	Loss: 0.696330
Train Epoch: 87 [19200/50000 (38%)]	Loss: 1.064911
Train Epoch: 87 [22400/50000 (45%)]	Loss: 0.574040
Train Epoch: 87 [25600/50000 (51%)]	Loss: 0.984248
Train Epoch: 87 [28800/50000 (58%)]	Loss: 0.842234
Train Epoch: 87 [32000/50000 (64%)]	Loss: 0.935412
Train Epoch: 87 [35200/50000 (70%)]	Loss: 1.111243
Train Epoch: 87 [38400/50000 (77%)]	Loss: 0.687848
Train Epoch: 87 [41600/50000 (83%)]	Loss: 1.066810
Train Epoch: 87 [44800/50000 (90%)]	Loss: 0.577195
Train Epoch: 87 [48000/50000 (96%)]	Loss: 0.915785

Test set: Average loss: 0.0075, Accuracy: 6766/10000 (68%)

Train Epoch: 88 [0/50

In [0]:
# if in Google Colab, download your model with this
# from google.colab import files
# files.download("best.pt")